In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os
from PIL import Image

In [2]:
model = load_model('mask.h5')

In [3]:
print(model.input_shape)
print(model.input_dtype)

(None, 224, 224, 3)
float32


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1,4)
    return faces

def preprocess_face(frame, face_box, target_size=(224, 224)):
    x, y, w, h = face_box
    face_roi = frame[y:y+h, x:x+w]  # Crop face
    face_resized = cv2.resize(face_roi, target_size)  # Resize to model input
    face_normalized = face_resized / 255.0  # Normalize
    return np.expand_dims(face_normalized, axis=0)  # Add batch dim

old version

In [6]:
# Initialize video capture (0 for default webcam)
cap = cv2.VideoCapture(0)

# Set desired dimensions (should match model input size)
frame_width = 224
frame_height = 224

def preprocess_frame(frame):
    # Resize to model's expected input
    frame = cv2.resize(frame, (frame_width, frame_height))
    
    # Convert BGR to RGB (OpenCV uses BGR by default)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Normalize pixel values (same as during training)
    frame = frame.astype('float32') / 255.0
    
    # Add batch dimension
    frame = np.expand_dims(frame, axis=0)
    
    return frame


# Class labels (modify according to your model)
class_labels = ['withoutmask', 'mask']  # Example binary classification

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = preprocess_frame(frame)
    
    
    # Create a copy for display
    display_frame = frame.copy()
    
    # Preprocess the frame
    processed_frame = preprocess_frame(frame)
    
    predictions = model.predict(processed_frame)
    predicted_class = np.argmax(predictions[0])
    confidence = np.max(predictions[0])

    confidence_threshold = 0.7

    if confidence > confidence_threshold:
        # High confidence detection
        color = (0, 255, 0)  # Green
    else:
        # Low confidence
        color = (0, 0, 255)  # Red


    label = f"{class_labels[predicted_class]}: {confidence:.2f}"
    cv2.putText(display_frame,label,(10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1,color, 2)
    
    
    # Show the frame
    cv2.imshow('Real-time Detection', display_frame)
    
    # Break loop with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print(predictions)
print(predicted_class)

# Release resources
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━

In [8]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    faces = detect_faces(frame)
    
    for (x, y, w, h) in faces:
        # Draw bounding box (optional)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Preprocess and predict
        face_input = preprocess_face(frame, (x, y, w, h))
        prediction = model.predict(face_input)
        predicted_class = np.argmax(prediction[0])
        
        class_labels = ['withoutmask', 'mask']

        if predicted_class > 0:
            color = (0, 255, 0)  # Green
        else:
            color = (0, 0, 255)  # Red

        # Display prediction
        label = class_labels[predicted_class]
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
    cv2.imshow("Face Tracking + CNN", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━